In [110]:
# old query
from bs4 import BeautifulSoup
from requests_html import HTMLSession

requests = HTMLSession()             # instantiate the HTMLSession Object, this returns executed JS code hence return html is the same as that of the page
                                     # with the requests library, via requests.get(url), that was not the case. The query randomly returns either run or unrun html

In [107]:
# new html query

# using selenium to scrape dynamic website
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import requests_html
import pandas as pd
import os
from bs4 import BeautifulSoup 
from requests_html import HTMLSession
from time import sleep

In [55]:
# define the browser and add the -- headless argument

chromedriver_path = os.getcwd() + '\\chromedriver\\chromedriver.exe'

opts = Options()
# opts.headless = True
opts.add_argument(' — headless')
driver = webdriver.Chrome(chromedriver_path, options=opts)

url  = 'https://ph.hotels.com/search.do?q-destination=Philippines&destination-id=10233139&as-shown=false&q-localised-check-in=06%2F04%2F2021&q-check-in=2021-04-06&q-localised-check-out=07%2F04%2F2021&q-check-out=2021-04-07&q-rooms=1&q-room-0-adults=2&q-room-0-children=0&intlid=modular%20country%20landing%20%3A%3A%20SQM'     
driver.maximize_window()
driver.get(url)                   # open the url, the process is automated

In [57]:
# the webpage is dynamic, continously scrolling down adds 12 more hotel cards in the results-list
# in order to gather enough data, we recursively execute a scroll down script

# with init at i = 1 with 12 hotel cards, keep scrolling
i = 1

while i <= 84: 
    # this adds 12 more hotel cards per iteration, if i > 84, the script stops, that is we end up with 1008 ...
    # ... hotel cards
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight,)")    # scroll down to the bottom
    print('webpage currently has {0} hotel cards'.format(i*12))
    print('loading next batch')
    i += 1
    sleep(20)   # gives time for the second scrolldown, adjust according to net speed.
                # faster net merits smaller sleep times
        
print('done')

webpage currently has 12 hotel cards
webpage currently has 24 hotel cards
webpage currently has 36 hotel cards
webpage currently has 48 hotel cards
webpage currently has 60 hotel cards
webpage currently has 72 hotel cards
webpage currently has 84 hotel cards
webpage currently has 96 hotel cards
webpage currently has 108 hotel cards
webpage currently has 120 hotel cards
webpage currently has 132 hotel cards
webpage currently has 144 hotel cards
webpage currently has 156 hotel cards
webpage currently has 168 hotel cards
webpage currently has 180 hotel cards
webpage currently has 192 hotel cards
webpage currently has 204 hotel cards
webpage currently has 216 hotel cards
webpage currently has 228 hotel cards
webpage currently has 240 hotel cards
webpage currently has 252 hotel cards
webpage currently has 264 hotel cards
webpage currently has 276 hotel cards
webpage currently has 288 hotel cards
webpage currently has 300 hotel cards
webpage currently has 312 hotel cards
webpage currently ha

In [112]:
page = driver.page_source    # get html format of the webpage at 1008 hotel cards
soup = BeautifulSoup(page, 'html.parser')


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=50389): Max retries exceeded with url: /session/36a168c55c3917e8b24a7459f5597f92/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000023F5C3488E0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [68]:
# hotel_cards :: BeautifulSoup Object per hotel card in site, with tag li
hotel_cardsBS = soup.select_one('body .resp-section .inner-section .resp-row .resp-col .main-inner section .h-listing .listings')   # BeautifulSoup object of containing all cards

bs4.element.Tag

In [69]:
hotel_cards = hotel_cardsBS.find_all('li', recursive=False) # creates one BS obj per card from hotel card BeautifulSoup
print('we have {0} distinct hotels'.format(len(hotel_cards)))

we have 620 distinct hotels


<h3> <b> Inspecting one of the hotel cards </b> </h3>
Upon inspection the ff were idd.
<li> an a tag with further data given by the attribute href
<li> address: in _2oHhXM contains address with format: specific, town/city, zip, province, Philippines 
<li> amenities: in ?
<li> price: in ? #rate is one day for 2 persons
<li> h ref has it all, must go to href for number of rooms
<blockquote>
    <li> number of rooms in:
    <li> customer reviews in:
    <li> add landmarks, and getting around: in 
</blockquote>

In [95]:
# try getting information from 1 hotel card
# proof of concept
hotel_sample = hotel_cards[1]

In [96]:
hotel_name = hotel_sample['data-title']
hotel_name


'Citadines Millennium Ortigas Manila'

In [97]:
rating = hotel_sample.find('strong',class_='guest-reviews-badge').get_text()
rating

'Superb 9.2'

In [98]:
amenities     = hotel_sample.select('.hmvt8258-amenities')
amenities_col = [amenity.get_text() for amenity in amenities[0].select('li')]
amenities_col

['Pool',
 'Parking available',
 'Pet-friendly',
 'Airport transfer',
 'Gym',
 'Restaurant']

In [100]:
address = hotel_sample.find('span', class_='address').get_text()
address

'11 Ortigas Avenue, Pasig, 01605, Philippines'

In [101]:
price = hotel_sample.find('a', class_='price-link').get_text()
price   # preprocess mo pa to

'P5,020'

In [105]:
href = hotel_sample.find('a', class_='property-name-link')['href']
href_url = 'https://ph.hotels.com'+ href

In [108]:
# open the page and get the ff
# number of roms
# customer review
# landmarks

requests = HTMLSession()

page_href = requests.get(href_url)
soup_href = BeautifulSoup(page_href.content, 'html.parser')

### working up to this point, the html attributes are different here | much easier to read

In [109]:
# class _1O0SKi has landmarks and get around         # get BS4 sub cats of this 2
# class _1O0SKi with id info5 has hotel size

body_href = soup_href.select_one('body div#root ._24UiGk ._3umLRC ._1rVrVi')
# body_href, works

In [110]:
hotel_size = soup_href.find(id='info5', class_='_1O0SKi')   # this is a bs4 object containing which has a child containing num of rooms
# info_href, works

room_count = hotel_size.select('._1_22qw ._2Ut1Ms ul li')[0]
room_count

AttributeError: 'NoneType' object has no attribute 'select'

In [101]:
# landmarks 
lndmrk_trnspo = soup_href.select_one('._1O0SKi')   # has children describing landmarks and transportation available

In [102]:
landmarks_bs4 = lndmrk_trnspo.select('ul li')      # the landmarks nearest to the place
landmarks = [landmark.get_text() for landmark in landmarks_bs4]

In [62]:
# transpo is further divided into categories
# pre process transpo
transpo_categories = lndmrk_trnspo.select('._2Ut1Ms div ul')      # outputs all categories and its transportation subs, one item is a category plus the transpo involved

# a transpo extraction concept
t_samp = transpo_categories[0]
t_samp

<ul class="shuttle"><li>Airport shuttle (surcharge)</li></ul>

In [60]:
t_class = t_samp['class']


for transpo_hub in t_samp:
    transpo_hub = t_samp.find('li').get_text()


In [16]:
'''
print(
'''
hotel name                  | 
address                     |
rating                      |
amneties                    |
number of rooms             |
price (2 person/ 1 day)     |
reviews                     |
landmarks                   |
'''
).format(
    hotel_name,
    rating,
    amneties,
    room_count,
    price,
    reviews,
    landmarks)
'''

SyntaxError: invalid syntax (<ipython-input-16-ecc34dba13a1>, line 4)